In [ ]:
import xlwings as xw
from xlwings import view

import pandas as pd

In [ ]:
import json,os
import read

from pprint import pprint
res_file=''##res_file_path


with open('test.json') as file:
    data = json.load(file)
with open(res_file) as file:
    data_pre = json.load(file)
fail_list_all=[]
success_list_all=[]

In [ ]:
import os
import json

def read_json_files_in_folder(folder_path):
    json_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, "r") as json_file:
                try:
                    data = json.load(json_file)
                    json_data.append(data)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON in file '{file_path}': {e}")
                except Exception as e:
                    print(f"Error reading file '{file_path}': {e}")
    return json_data

if __name__ == "__main__":
    folder_path = "lunwen_result"
    json_data_list   = read_json_files_in_folder(folder_path)
    print("Number of JSON files read:", len(json_data_list))
    # Do further processing with the json_data_list if needed


In [ ]:
concatenated_data = json.dumps(json_data_list)

In [ ]:
data_test=[]
for item in json_data_list:
    data_test+=item
data_pre=data_test

In [ ]:

data_pref=[data_pre[i] for i in range(len(data_pre))
                            if data_pre[i]['level']!='Calculation']
data_prec=[data_pre[i] for i in range(len(data_pre))
                            if data_pre[i]['level']=='Calculation']
data_p=data_pref+data_prec
result=[]
for j in range(len(data)):
    for i in range(len(data[j]['t5Formulas'])):
        data[j]['t5Formulas'][i]['Table']=data[j]['Table']
        result.append(data[j]['t5Formulas'][i])
        
len(result),len(data_p)

In [ ]:

data_p_dict = {item['question'].split(" |")[0]: index for index, item in enumerate(data_p)}
x=0
test_list=[]
for i in range(len(result)):
    question = result[i]['Question']
    if question in data_p_dict:
        data_index = data_p_dict[question]
        result[i]['pre_f'] = data_p[data_index]['prediction']
        test_list.append(result[i])
        x+=1
        # print(x)
    else:
        print(f"No matching context found for question: {question}")
len(test_list)



In [ ]:
with open('gpt_eval_3000.json','w') as file:
    json.dump(random_elements,file)

In [ ]:
with open('gpt_eval_3000.json','r') as file:
    data=json.load(file)

In [ ]:
os.environ['OPENAI_API_KEY'] = read.read_json(os.path.join('configuration.json'))['OpenAI_api_key']


In [ ]:
read.read_json(os.path.join('configuration.json'))['OpenAI_api_key']

In [ ]:
def get_formular(question: str,table, model='gpt-3.5-turbo'):
    llm = ChatOpenAI(model_name=model, max_tokens=50,)
    prompt = "Write a excel formula to answer the quesion based on table beleow\n"\
        "You can use functions like SUM,XLOOKUP,UNIQUE,AVERAGE,etc...\n"\
             "Question:{question}\n" \
             "Here is the Table: {table}.\n" \
             "{format_instructions}"
             
    prompt_template = ChatPromptTemplate.from_template(prompt)
    
    #options = ResponseSchema(name="words", description="The words are related to bad behavior.")
    options = ResponseSchema(name="Formula", description="The formula is related to question")
    response_schemas = [options]
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    form_instructions = output_parser.get_format_instructions()
    
    final_prompt = prompt_template.format_messages(question=question, table=table, format_instructions=form_instructions)
    print(final_prompt)
    output = llm(final_prompt)
    return output_parser.parse(output.content)

In [ ]:
max_iterations = 5000  # Set a maximum number of iterations to avoid infinite loop

for iteration in range(max_iterations):
    try:
        for i, item in enumerate(data[:]):
            # if 'llmFormula' not in item:
            if 1:
                print(i)
                llmout = get_formular(item["Question"], item["Table"])
                item['llmFormula'] = llmout['Formula']
    except Exception as e:
        print(f"An error occurred: {e}")
    else:
        # The loop completed without errors, break out of the while loop
        break
else:
    print("Maximum number of iterations reached, exiting loop.")


In [ ]:
def eval_execution(sheet,item,table,f1:str,f2:str,):
    f_cell='A30'
    f_cell2='A31'
    save_table(sheet,table)
    if sheet.range(f_cell).value is None:
        sheet.range(f_cell).formula =f"={f1}"          
        sheet.range(f_cell2).formula =f"={f2}"
        
        result1 = sheet.range(f_cell).value
        result2 = sheet.range(f_cell2).value
        print(f1,"\n",f2,"\n\n")
        
        if result1 is None:
            return False,result1,result2
        if result1 is not None and result2 is not None:
            if result1==result2:
                item['exe_results']=result1

                return True,result1,result2

        else:
            print('Fail to get the answer')   
    else:
        raise ValueError("Cell not Empty!")
    return False,result1,result2

def run_execution_test(test_list,sheet=None):
    count,count_true=0,0
    em,exe=0,0
    count_wronggd=0
    fail_list=[]
    #count_true_em,count_true_exe=0,0
    for item in test_list[:]:
        item['res']="failed"
        pre_f = item.get('pre_f')
        if pre_f is None:
            item['res']='not found'
            continue
        count+=1
        formula2 = item['Formula2']
        a=formula2.lower().replace(" ", "").replace("\"", "")
        b=pre_f.lower().replace(" ", "").replace("\"", "")
        nl=item['Question']
        if a==b:   
            item['res']='exact_match'
            count_true+=1
            em+=1
            #print(formula2,"\n",pre_f)
            print(count,'exact_match')
        else:
            flag,gdres,llmres=eval_execution1(sheet,item,item['Table'],formula2,pre_f)
            if(flag):
                exe+=1
                item['res']='exe_match'
                count_true+=1
    print("em",em)
    print("exe",exe)
    return fail_list,count_true/count,count_wronggd

In [ ]:
from utils import *
import json

app = xw.App(visible=True)


book = app.books.open(r"b1.xlsx")
sheet = book.sheets[0] 

fail_list,percentage,wg=run_execution_test(data,sheet)
print(percentage)

book.close()
app.quit()

In [ ]:
res_list=test_list

In [ ]:
import pandas as pd

counters = {
    'easy': {'exact_match': 0, 'exe_match': 0,'failed': 0},
    'medium': {'exact_match': 0, 'exe_match': 0,'failed': 0},
    'hard': {'exact_match': 0, 'exe_match': 0,'failed': 0},
    'Calculation': {'exact_match': 0, 'exe_match': 0,'failed': 0},
}


for item in res_list:
    if item is None:
        continue
    level = item['Level']
    result = item['res']

    counters[level][result] += 1

for level, results in counters.items():
    print(f"Level: {level}")
    total = sum(results.values())
    for result, count in results.items():
        percentage = (count / total) * 100
        print(f"{result}: {count} ({percentage:.2f}%)")
        
df_res_l=pd.DataFrame(counters).T
df_res_l


In [ ]:

import importlib,utils,json,os
from utils import *
gpt_eval=utils.read_json("gpt_eval.json")


In [ ]:

counters = {
    'easy': {'exact_match': 0, 'exe_match': 0,'failed': 0},
    'medium': {'exact_match': 0, 'exe_match': 0,'failed': 0},
    'hard': {'exact_match': 0, 'exe_match': 0,'failed': 0},
    'Calculation': {'exact_match': 0, 'exe_match': 0,'failed': 0},
}


for item in res_list:
    level = item['Level']
    result = item['res']
    # if(result!="failed"):
    #     print(result)
    counters[level][result] += 1


for level, results in counters.items():
    print(f"Level: {level}")
    total = sum(results.values())
    for result, count in results.items():
        percentage = (count / total) * 100
        print(f"{result}: {count} ({percentage:.2f}%)")
        
df_res_l=pd.DataFrame(counters).T
df_res_l

df_res_l["em_rate"]=df_res['exact_match']/(df_res['exact_match']+df_res["failed"])
print(df_res_l['exact_match'].sum()/(df_res_l['exact_match'].sum()+df_res_l['failed'].sum()))
df_res_l
